In [208]:
import numpy as np
import matplotlib.pyplot as plt
import keras

In [209]:
N = 3

In [210]:
from keras.datasets import mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [ ]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

In [ ]:
x_train = x_train /255.0
x_test = x_test /255.0
x_validation = x_train[:6000]
y_validation = y_train[:6000]
x_train = x_train[6000:]
y_train = y_train[6000:]

x_validation = x_validation.reshape(x_validation.shape[0],-1).T
x_train = x_train.reshape(x_train.shape[0],-1).T
x_test = x_test.reshape(x_test.shape[0],-1).T


print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
#print(x_validation.shape,y_validation.shape)

In [213]:
def one_hot(Y):
  one_hot_Y = np.zeros((Y.size,Y.max()+1))
  one_hot_Y[np.arange(Y.size),Y] = 1
  one_hot_Y = one_hot_Y.T
  return one_hot_Y

In [ ]:
print(y_test)
print(y_train)
print(x_train.shape)
print(y_train.shape)

In [215]:
def init_parameter(N):
  d = [10] * (N+1)
  for i in range(N):
    d[i] = 2 ** (N-i+5)
  d[0] = 784
  print(d)
  W = []
  b = []
  for i in range(N):
    W.append(np.random.randn(d[i],d[i+1]))
    b.append(np.random.randn(d[i+1],1))
  return W,b

def RELU(Z):
  return np.maximum(0,Z)


def softmax(Z):
    Z_max = np.max(Z, axis=0, keepdims=True)
    exp_Z = np.exp(Z - Z_max)
    A = exp_Z / np.sum(exp_Z, axis=0, keepdims=True)
    return A

def one_hot(Y):
  one_hot_Y = np.zeros((Y.size,10))
  one_hot_Y[np.arange(Y.size),Y] = 1
  one_hot_Y = one_hot_Y.T
  return one_hot_Y

def prop(W,b,X,N):
  Z = []
  A = []
  for i in range(N):
    if i == 0 :
      Z.append(W[0].T.dot(X) + b[0])
      A.append(RELU(Z[0]))
    elif i != N-1 :
      Z.append(np.dot(W[i].T, A[i-1]) + b[i])
      A.append(RELU(Z[i]))
    else:
      Z.append(np.dot(W[i].T, A[i-1]) + b[i])
      A.append(softmax(Z[i]))
  return Z,A



def back_prop(Z,A,W,X,Y,N):
  E = []
  dW = []
  db = []
  one_hot_Y = one_hot(Y)
  for i in range(N-1, -1, -1):
      if i == N-1:
          E.append(1/Y.size * (A[-1] - one_hot_Y))
          dW.insert(0, np.dot(A[i-1], E[-1].T))
      elif i == 0:
          E.append(np.dot(W[i+1], E[-1]))
          E[-1][Z[0] <= 0] = 0
          dW.insert(0, np.dot(X, E[-1].T))
      else:
          E.append(np.dot(W[i+1], E[-1]))
          E[-1][Z[i] <= 0] = 0
          dW.insert(0, np.dot(A[i-1], E[-1].T))

      db.insert(0, np.sum(E[-1], axis=1, keepdims=True))
  return dW, db
def update_parameter(W,b,dW,db,eta):
  for i in range(len(W)):
    W[i] = W[i] - eta * dW[i]
    b[i] = b[i] - eta * db[i]
  return W,b
def cost(Y, Yhat):
  return -np.sum(Y * np.log(Yhat+1e-6))/Y.shape[1]

In [216]:
def get_predictions(Yhat):
  return np.argmax(Yhat,0)
def get_accuracy(predictions,Y):
  return np.sum(predictions == Y) / Y.size

In [217]:
#N là số layer
def train(N,W,b,X,Y,eta,epochs):
  batch = 64
  for i in range(epochs):
    for j in range(0,X.shape[1],batch):
      X_batch = X[:,j:j+batch]
      Y_batch = Y[j:j+batch]
      one_hot_Y = one_hot(Y_batch)
      Z,A = prop(W,b,X_batch,N)
      dW,db = back_prop(Z,A,W,X_batch,Y_batch,N)
      W,b = update_parameter(W,b,dW,db,eta)
      if i%10 == 0 and j == 0:
        print(i, "cost: ", cost(one_hot_Y,A[-1]), "Accuracy train:", get_accuracy(get_predictions(A[-1]),Y_batch)*100,"% ", "Accuracy validation: ",get_accuracy(test(N,W,b,x_test,y_test),y_test)*100, "%")
  return W,b

def test(N,W,b,X,Y):
    one_hot_Y = one_hot(Y)
    Z,A = prop(W,b,X,N)
    return get_predictions(A[-1])

In [ ]:
W,b = init_parameter(N)

In [ ]:
W,b = train(N,W,b,x_train,y_train,0.01,501)

In [ ]:
print("Accuracy test: ",get_accuracy(test(N,W,b,x_test,y_test),y_test)*100, "%")